## Converting to Supergalactic Coordinates ##

we describe how we convert from RA, DEC, and redshift to SGX,SGY, and SGZ.

We try to reproduce the figures shown in Kim+2016

### load modules ###

In [1]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import os
import sys
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
from mpl_toolkits.mplot3d import Axes3D
from astropy.cosmology import WMAP9 as cosmo
from astropy.constants import c
%matplotlib inline

//anaconda/envs/astroconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Read in NSA catalog ###

In [3]:
#Get current path so program can tell if this is being run on Kelly's or Rose's computer
mypath=os.getcwd()
if mypath.find('rfinn') > -1:
    print "Running on Rose's computer"
    #agcfile='/Users/rfinn/idl/programs/idl_alfa/agctotal.sav'
    gitpath='/Users/rfinn/github/'
elif mypath.find('kelly') > -1:
    print "Running on Kellys's computer"
    gitpath='/Users/kellywhalen/Github/'


#### READ IN DATA #####

#NSA Table
NSA_file = gitpath+'Virgo/tables/nsa_v0_1_2.fits'
#NSA_file='/Users/rfinn/research/nsa/nsa_v0_1_2.fits'
nsa = fits.getdata(NSA_file)
nsa = nsa[0:21]
nsadict=dict((a,b) for a,b in zip(nsa.NSAID,np.arange(len(nsa.NSAID))))

Running on Rose's computer


### Select Galaxies in the vicinity of Virgo ###

In [52]:
raflag = (nsa.RA > 115.) & (nsa.RA < 240.)
decflag= (nsa.DEC > -30.) & (nsa.DEC < 60.)
velflag =  (nsa.Z*3.e5 < 3000.) # & (nsa.Z*3.e5 > 1000.) 
vflag = raflag & decflag & velflag 

#nsa = nsa[vflag]

# define nsa RA and Dec as SkyCoord
nsa_sc = SkyCoord(nsa.RA*u.degree, nsa.DEC*u.degree)


##  FLOW MODEL  ##

convert RA and DEC to galactic
* l = galcoords.l.degree
* b = galcoords.b.degree



using mould+00 method to correct redshifts described in detail in appendix A

http://adsabs.harvard.edu/abs/2000ApJ...529..786M


### 1. Correct to centroid of the Local Group ###

In [53]:
# convert helio-centric velocity to units of km/s
V_H = nsa.Z*c.to('km/s')

# 1. correction of observed heliocentric velocity to centroid of local group
# VLG = VH - 79 cos l cos b + 296 sin l cos b - 36 sin b

V_LG = - 79.*u.km/u.second*np.cos(nsa_sc.galactic.l.radian)*np.cos(nsa_sc.galactic.b.radian) + 296.*u.km/u.second*np.sin(nsa_sc.galactic.l.radian)*np.cos(nsa_sc.galactic.b.radian) - 36.*u.km/u.second*np.sin(nsa_sc.galactic.b.radian)


### 2. Correction for Virgo infall

In [54]:
# from Mould+2000, ApJ, 529, 786
# not sure if we are using this equation exactly right

V_fid = 200.*u.km/u.second # infall of LG into Virgo?
# Virgo coordinates given by Mould+2000
Virgo = SkyCoord('12h28m19s', '+12d40m00s', frame='fk5',equinox='J1950.') # epoch = 1950
Virgo = Virgo.transform_to(FK5(equinox='J2000'))
### need to fix this to use spherical distance
#theta = np.sqrt((nsa_sc.ra.radian - Virgo.ra.radian)**2 + (nsa_sc.dec.radian - Virgo.dec.radian)**2)
theta = Virgo.separation(nsa_sc).radian 
# cluster radius in deg
gamma = 2.

V_a = 1035.*u.km/u.second # recession vel of Virgo from Mould+2000
#V_a = 1221.*u.km/u.second # recession vel of Virgo as calculated by Gianluca
V_o = nsa.Z*c.to('km/s') + V_LG # recession velocities of the galaxies
r_oa = np.sqrt(V_o**2 + V_a**2 - 2.*V_o*V_a*np.cos(theta))
V_infall = V_fid*(np.cos(theta) + (V_o - V_a*np.cos(theta))/r_oa*(r_oa/V_a)**(1-gamma))

### 3. Correction for GA infall ###

In [55]:
V_fid = 400.*u.km/u.second # infall of LG into Virgo?
# GA coordinates given by Mould+2000
GA = SkyCoord('13h20m00s', '+44d00m00s', frame='fk5',equinox='J1950.') # epoch = 1950
GA = GA.transform_to(FK5(equinox='J2000'))
theta = GA.separation(nsa_sc).radian
#theta = np.sqrt((nsa_sc.ra.radian - GA.ra.radian)**2 + (nsa_sc.dec.radian - GA.dec.radian)**2)

gamma = 2.
V_a = 4380.*u.km/u.second # recession vel of Great Attractor from Mould+2000
V_o = nsa.Z*c.to('km/s') + V_LG # recession velocities of the galaxies
r_oa = np.sqrt(V_o**2 + V_a**2 - 2.*V_o*V_a*np.cos(theta))
V_GA = V_fid*(np.cos(theta) + (V_o - V_a*np.cos(theta))/r_oa*(r_oa/V_a)**(1-gamma))

### 4. Correction for Shapley supercluster infall. ###

In [56]:
# Final, correction cosmic velocity is
#
#  Vcosmic = VH + Vc,LG + Vin,Virgo + Vin,GA + Vin,Shap + ...

# ## Tranforming to Supergalactic Coordinates##
# 
# Looking to match the plots shown in Kim+2016
# 
# https://arxiv.org/abs/1611.00437

### Add up all of the corrections ###

In [57]:
#Plot of Virgo Cluster in galactic coordinates
V_cosmic = V_H + V_LG + V_infall +V_GA
V = V_cosmic

### Convert to Supergalactic X, Y, and X coordinates###

In [58]:
#SGX = V/cosmo.H(0)*np.cos(nsa_sc.supergalactic.sgl.radian)*np.cos(nsa_sc.supergalactic.sgb.radian)
#SGY = V/cosmo.H(0)*np.sin(nsa_sc.supergalactic.sgl.radian)*np.cos(nsa_sc.supergalactic.sgb.radian)
#SGZ = V/cosmo.H(0)*np.sin(nsa_sc.supergalactic.sgb.radian)


# using H0 = 74 to match Kim+2016 paper
#H0 = 74.*u.km/u.second/u.Mpc
H0 = 100.*u.km/u.second/u.Mpc #Kim's email
SGX = V/H0*np.cos(nsa_sc.supergalactic.sgl.radian)*np.cos(nsa_sc.supergalactic.sgb.radian)
SGY = V/H0*np.sin(nsa_sc.supergalactic.sgl.radian)*np.cos(nsa_sc.supergalactic.sgb.radian)
SGZ = V/H0*np.sin(nsa_sc.supergalactic.sgb.radian)

### Subtract the Position of Virgo ###

In [59]:
distance_to_virgo = 16.5*u.Mpc #Mpc
SGX_Virgo = distance_to_virgo*np.cos(Virgo.supergalactic.sgl.radian)*np.cos(Virgo.supergalactic.sgb.radian)
SGY_Virgo = distance_to_virgo*np.sin(Virgo.supergalactic.sgl.radian)*np.cos(Virgo.supergalactic.sgb.radian)
SGZ_Virgo = distance_to_virgo*np.sin(Virgo.supergalactic.sgb.radian)

# difference 
DSGX = SGX - SGX_Virgo
DSGY = SGY - SGY_Virgo
DSGZ = SGZ - SGZ_Virgo


### Various cuts used by Kim+2016 ###

In [60]:
# don't use objects w/in 3.6 Mpc of Virgo (Kim+2016)http://localhost:8888/notebooks/VirgoCatalog.ipynb#

dist = np.sqrt(DSGX**2 + DSGY**2 + DSGZ**2) 
dist = dist / u.Mpc

distarray = np.array(dist)
vdistflag = distarray < 50

VirgoTestSet = nsa[vdistflag]


In [61]:
def VirgoCatalog(NSA_file):

    #Load in all packages
    import numpy as np
    from astropy.io import fits
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    import os
    import sys
    from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
    from astropy.coordinates import Angle, Latitude, Longitude  # Angles
    from mpl_toolkits.mplot3d import Axes3D
    from astropy.cosmology import WMAP9 as cosmo
    from astropy.constants import c
    
    #read in .fits file
    nsa = fits.getdata(NSA_file)
    nsadict=dict((a,b) for a,b in zip(nsa.NSAID,np.arange(len(nsa.NSAID))))
    
    #Pick out galaxies in the general region of virgo
    raflag = (nsa.RA > 115.) & (nsa.RA < 240.)
    decflag= (nsa.DEC > -30.) & (nsa.DEC < 60.)
    velflag =  (nsa.Z*3.e5 < 3000.) 
    vflag = raflag & decflag & velflag 

    # define nsa RA and Dec as SkyCoord
    nsa_sc = SkyCoord(nsa.RA*u.degree, nsa.DEC*u.degree)
    
    # convert helio-centric velocity to units of km/s
    V_H = nsa.Z*c.to('km/s')

    # 1. correction of observed heliocentric velocity to centroid of local group
    # VLG = VH - 79 cos l cos b + 296 sin l cos b - 36 sin b

    V_LG = - 79.*u.km/u.second*np.cos(nsa_sc.galactic.l.radian)*np.cos(nsa_sc.galactic.b.radian) + 296.*u.km/u.second*np.sin(nsa_sc.galactic.l.radian)*np.cos(nsa_sc.galactic.b.radian) - 36.*u.km/u.second*np.sin(nsa_sc.galactic.b.radian)

    # from Mould+2000, ApJ, 529, 786
    # not sure if we are using this equation exactly right

    V_fid = 200.*u.km/u.second # infall of LG into Virgo?
    # Virgo coordinates given by Mould+2000
    Virgo = SkyCoord('12h28m19s', '+12d40m00s', frame='fk5',equinox='J1950.') # epoch = 1950
    Virgo = Virgo.transform_to(FK5(equinox='J2000'))
    ### need to fix this to use spherical distance
    #theta = np.sqrt((nsa_sc.ra.radian - Virgo.ra.radian)**2 + (nsa_sc.dec.radian - Virgo.dec.radian)**2)
    theta = Virgo.separation(nsa_sc).radian 
    # cluster radius in deg
    gamma = 2.

    V_a = 1035.*u.km/u.second # recession vel of Virgo from Mould+2000
    #V_a = 1221.*u.km/u.second # recession vel of Virgo as calculated by Gianluca
    V_o = nsa.Z*c.to('km/s') + V_LG # recession velocities of the galaxies
    r_oa = np.sqrt(V_o**2 + V_a**2 - 2.*V_o*V_a*np.cos(theta))
    V_infall = V_fid*(np.cos(theta) + (V_o - V_a*np.cos(theta))/r_oa*(r_oa/V_a)**(1-gamma))
    
    V_fid = 400.*u.km/u.second # infall of LG into Virgo?
    # GA coordinates given by Mould+2000
    GA = SkyCoord('13h20m00s', '+44d00m00s', frame='fk5',equinox='J1950.') # epoch = 1950
    GA = GA.transform_to(FK5(equinox='J2000'))
    theta = GA.separation(nsa_sc).radian
    #theta = np.sqrt((nsa_sc.ra.radian - GA.ra.radian)**2 + (nsa_sc.dec.radian - GA.dec.radian)**2)

    gamma = 2.
    V_a = 4380.*u.km/u.second # recession vel of Great Attractor from Mould+2000
    V_o = nsa.Z*c.to('km/s') + V_LG # recession velocities of the galaxies
    r_oa = np.sqrt(V_o**2 + V_a**2 - 2.*V_o*V_a*np.cos(theta))
    V_GA = V_fid*(np.cos(theta) + (V_o - V_a*np.cos(theta))/r_oa*(r_oa/V_a)**(1-gamma))
    
    #Add up all corrections
    V_cosmic = V_H + V_LG + V_infall +V_GA
    V = V_cosmic
    
    #Convert to supergalactic coordinates
    # using H0 = 74 to match Kim+2016 paper
    #H0 = 74.*u.km/u.second/u.Mpc
    H0 = 100.*u.km/u.second/u.Mpc #Kim's email
    SGX = V/H0*np.cos(nsa_sc.supergalactic.sgl.radian)*np.cos(nsa_sc.supergalactic.sgb.radian)
    SGY = V/H0*np.sin(nsa_sc.supergalactic.sgl.radian)*np.cos(nsa_sc.supergalactic.sgb.radian)
    SGZ = V/H0*np.sin(nsa_sc.supergalactic.sgb.radian)
    
    #Getting distance from virgo
    distance_to_virgo = 16.5*u.Mpc #Mpc
    SGX_Virgo = distance_to_virgo*np.cos(Virgo.supergalactic.sgl.radian)*np.cos(Virgo.supergalactic.sgb.radian)
    SGY_Virgo = distance_to_virgo*np.sin(Virgo.supergalactic.sgl.radian)*np.cos(Virgo.supergalactic.sgb.radian)
    SGZ_Virgo = distance_to_virgo*np.sin(Virgo.supergalactic.sgb.radian)

    # difference 
    DSGX = SGX - SGX_Virgo
    DSGY = SGY - SGY_Virgo
    DSGZ = SGZ - SGZ_Virgo

    dist = np.sqrt(DSGX**2 + DSGY**2 + DSGZ**2) 
    dist = dist / u.Mpc
    
    distarray = np.array(dist)
    vdistflag = distarray < 50 #What is the radius of virgo??

    VirgoTestSet = nsa[vdistflag]

    fits.writeto('/Users/kellywhalen/Github/Virgo/tables/VirgoCatalog.fits', VirgoTestSet, clobber=True)

In [62]:
table_ext = '/Users/kellywhalen/Github/Virgo/tables/'

In [77]:
#Line Match with Jablonka Catalog

vdat = fits.getdata(table_ext +'VirgoCatalog.fits')
jdat = fits.getdata(table_ext + 'Jablonka-MasterFile.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
jcat = SkyCoord(jdat.RA*u.degree,jdat.DEC*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(jcat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'CO-HI_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=jdat.dtype)
matchedarray1[matchflag] = jdat[index[matchflag]]

fits.writeto(outfile,matchedarray1,clobber=True)
#t = fits.getdata(outfile)
#t.add_column()



AttributeError: 'numpy.ndarray' object has no attribute 'add_column'

In [64]:
#Line match this catalog to Moustakas's stellar mass file
vdat = fits.getdata(table_ext + 'VirgoCatalog.fits')
mdat = fits.getdata(table_ext + 'nsa_v1_2_fsps_v2.4_miles_chab_charlot_sfhgrid01.Virgo.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
masscat = SkyCoord(mdat.RA*u.degree,mdat.DEC*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(masscat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'StellarMasstoNSA_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=mdat.dtype)
matchedarray1[matchflag] = mdat[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)

In [65]:
#Line match this catalog to Yang Model
vdat = fits.getdata(table_ext + 'VirgoCatalog.fits')
ydat = fits.getdata(table_ext + 'YangDR7ModelToNSA.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
yangcat = SkyCoord(ydat.RA_gal*u.degree,ydat.DEC_gal*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(yangcat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'YangModeltoNSA_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=ydat.dtype)
matchedarray1[matchflag] = ydat[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)

In [66]:
#Line match this catalog to Yang Petro
vdat = fits.getdata(table_ext + 'VirgoCatalog.fits')
ydat = fits.getdata(table_ext + 'YangDR7PetroToNSA.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
yangcat = SkyCoord(ydat.RA_gal*u.degree,ydat.DEC_gal*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(yangcat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'YangPetrotoNSA_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=ydat.dtype)
matchedarray1[matchflag] = ydat[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)

In [67]:
#Line match this catalog to simard 1

vdat = fits.getdata(table_ext + 'VirgoCatalog.fits')
sdat = fits.getdata(table_ext + 'Simard1ToNSA.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
simardcat = SkyCoord(sdat._RA*u.degree,sdat._DE*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(simardcat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'Simard1toNSA_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=sdat.dtype)
matchedarray1[matchflag] = sdat[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)



In [68]:
#Line match this catalog to simard 2

vdat = fits.getdata(table_ext + 'VirgoCatalog.fits')
sdat = fits.getdata(table_ext + 'Simard2ToNSA.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
#simardcat = SkyCoord(sdat._RA*u.degree,sdat._DE*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(simardcat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'Simard2toNSA_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=sdat.dtype)
matchedarray1[matchflag] = sdat[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)

In [69]:
#Line match this catalog to simard 3

vdat = fits.getdata(table_ext + 'VirgoCatalog.fits')
sdat = fits.getdata(table_ext +'Simard3ToNSA.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
#simardcat = SkyCoord(sdat._RA*u.degree,sdat._DE*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(simardcat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile=table_ext + 'Simard3toNSA_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=sdat.dtype)
matchedarray1[matchflag] = sdat[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)

In [70]:
#Line match with GALEX/WISE
#ALL SKY CATALOG

#initializes record array
infile = table_ext + 'GSWLC-A1.dat'
catA1 = np.loadtxt(infile, dtype = {'names':('objID','glxID','plate','MJD','fiberID','RA','DEC','z','chisq','logMstar','err_logMstar','logSFR_SED','err_logSFR_SED','A_FUV','err_A_FUV','A_B','err_A_B','A_V','err_A_V','flag_SED','UV_survey','logSFR_MIR_AW','flag_wise','logSFR_MIR_UW','flag_unwise','flag_mgs'), 'formats': (np.int,# ObjID
                                   np.int,
                                   np.int,
                                   np.int,
                                   np.int,# fiberID
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float, #log SFR_SED
                                   np.float,
                                   np.float, #A_FUV
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.int, #flag_sed
                                   np.int, #UV surve
                                   np.float,
                                   np.int,
                                   np.float,
                                   np.int,
                                   np.int)})


#matching time!
virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
samircat = SkyCoord(catA1['RA']*u.degree,catA1['DEC']*u.degree,frame='icrs')
index,dist2d,dist3d = virgocat.match_to_catalog_sky(samircat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile = table_ext + 'GALEX-WISE-allsky_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=catA1.dtype)
matchedarray1[matchflag] = catA1[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)


In [71]:
#MID SKY

#initializes record array
infile = table_ext + 'GSWLC-M1.dat'
catA1 = np.loadtxt(infile, dtype = {'names':('objID','glxID','plate','MJD','fiberID','RA','DEC','z','chisq','logMstar','err_logMstar','logSFR_SED','err_logSFR_SED','A_FUV','err_A_FUV','A_B','err_A_B','A_V','err_A_V','flag_SED','UV_survey','logSFR_MIR_AW','flag_wise','logSFR_MIR_UW','flag_unwise','flag_mgs'), 'formats': (np.int,# ObjID
                                   np.int,
                                   np.int,
                                   np.int,
                                   np.int,# fiberID
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float, #log SFR_SED
                                   np.float,
                                   np.float, #A_FUV
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.int, #flag_sed
                                   np.int, #UV surve
                                   np.float,
                                   np.int,
                                   np.float,
                                   np.int,
                                   np.int)})


#matching time!
virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
samircat = SkyCoord(catA1['RA']*u.degree,catA1['DEC']*u.degree,frame='icrs')
index,dist2d,dist3d = virgocat.match_to_catalog_sky(samircat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'GALEX-WISE-mid_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=catA1.dtype)
matchedarray1[matchflag] = catA1[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)


In [72]:
#Line match with GALEX/WISE
#DEEP SKY CATALOG

#initializes record array
infile = table_ext + 'GSWLC-D1.dat'
catA1 = np.loadtxt(infile, dtype = {'names':('objID','glxID','plate','MJD','fiberID','RA','DEC','z','chisq','logMstar','err_logMstar','logSFR_SED','err_logSFR_SED','A_FUV','err_A_FUV','A_B','err_A_B','A_V','err_A_V','flag_SED','UV_survey','logSFR_MIR_AW','flag_wise','logSFR_MIR_UW','flag_unwise','flag_mgs'), 'formats': (np.int,# ObjID
                                   np.int,
                                   np.int,
                                   np.int,
                                   np.int,# fiberID
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float, #log SFR_SED
                                   np.float,
                                   np.float, #A_FUV
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.float,
                                   np.int, #flag_sed
                                   np.int, #UV surve
                                   np.float,
                                   np.int,
                                   np.float,
                                   np.int,
                                   np.int)})


#matching time!
virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
samircat = SkyCoord(catA1['RA']*u.degree,catA1['DEC']*u.degree,frame='icrs')
index,dist2d,dist3d = virgocat.match_to_catalog_sky(samircat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'GALEX-WISE-deepsky_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=catA1.dtype)
matchedarray1[matchflag] = catA1[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)



In [73]:
#Line Match with AGC catalog

vdat = fits.getdata(table_ext +'VirgoCatalog.fits')
agcdat = fits.getdata(table_ext + 'agctotal.fits')

virgocat = SkyCoord(vdat.RA*u.degree,vdat.DEC*u.degree,frame='icrs')
agccat = SkyCoord(agcdat.RA*u.degree,agcdat.DEC*u.degree,frame='icrs')

index,dist2d,dist3d = virgocat.match_to_catalog_sky(agccat)

# only keep matches with matched RA and Dec w/in 1 arcsec
matchflag = dist2d.degree < 2./3600

# write out line-matched catalog
outfile= table_ext + 'AGCtoNSA_virgo.fits'
matchedarray1=np.zeros(len(vdat),dtype=agcdat.dtype)
matchedarray1[matchflag] = agcdat[index[matchflag]]
fits.writeto(outfile,matchedarray1,clobber=True)